In [1]:
import requests

In [11]:
sys = """Objective: Create a detailed list of skills from the provided NOS and associate each skill with assessment criteria aligned to Bloom's Taxonomy (6 levels: Remember, Understand, Apply, Analyze, Evaluate, Create). This will ensure meaningful skill assessment for both technical and soft skills.

Instructions for the System:

Extract Skills:

Analyze the provided NOS document and extract all the skills mentioned. For each skill, ensure both technical (e.g., product knowledge, technical tools) and soft skills (e.g., communication, customer engagement) are included.
Each skill should be contextualized based on the job description (JD) and performance standards outlined in the NOS.
Define Assessment Criteria:

For each skill, generate assessment criteria based on the six levels of Bloom’s Taxonomy:
Remember: Does the user recall relevant facts, definitions, or procedures for this skill?
Understand: Can the user explain or interpret the concepts related to this skill?
Apply: Can the user use the skill in real-world scenarios or simulated tasks?
Analyze: Can the user break down complex situations to identify components related to this skill?
Evaluate: Can the user assess situations and justify decisions involving this skill?
Create: Can the user design or innovate new approaches, presentations, or solutions based on this skill?
Associate Performance and Knowledge Criteria:

Align each skill with performance standards and knowledge requirements from the NOS. This ensures the assessment is comprehensive and role-specific.
Include prompts for assessing both practical demonstration and theoretical understanding.
Output Format:

For each skill, provide:
Skill Name
Skill Category (Technical/Soft)
Bloom's Taxonomy Level Criteria:
Remember: Specific questions or tasks to test recall.
Understand: Scenarios to explain the skill.
Apply: Practical tasks or case studies.
Analyze: Problem-solving scenarios.
Evaluate: Judgment or decision-making tasks.
Create: Designing new solutions or improvements.
"""

In [12]:
system_message = {'role':'system','content':sys}
nos = """Performance criteria
You must be able to:
1. use sales-related information for planning and implementing sales activities
2. store information on the sales technology systems, checking its quality, validity
and reliability
3. maintain the sales-related information stored in the technology systems, ensuring
it is accurate and up to date
4. use tools and technology systems for automation of sales-related tasks and
activities
5. share sales-related information for collaboration with members of sales team
6. use lead generation automation technology to qualify the leads
7. collate potential leads and prioritise these in accordance with sales targets
8. summarise information appropriately from sales systems for reporting to
management
9. follow up incoming leads and assign the ownership of quality leads to the most
relevant members of sales team
10. monitor, process or delegate daily tasks within assigned ownership and visibility on
technology systems
11. monitor own use of the sales technology system, seeking guidance, support and
skills development, where necessary
12. provide feedback on the sales technology systems, your pipeline management and
other tasks and activities
13. seek evidence-based recommendations for improvement of sales activities and
personal productivity using the technology systems
INSSAL014
Use sales tools and technology systems
INSSAL014 Use sales tools and technology systems 2
Knowledge and understanding
You need to know and understand:
1. the current legal and ethical issues in relation to the use of sales tools and
technology systems
2. the benefits of tools and technology systems for sales-related tasks and activities
3. the range of tasks and activities on sales tools and technology systems
4. the visibility options and ownership of sales team members on technology systems
5. the information requirements for the sales function, lead generation and
qualification of these
6. your organisation’s sales targets
7. the importance of automation of sales-related tasks and activities
8. how to collate, qualify and prioritise the leads on technology systems
9. the sales pipeline and how to feed the leads into it using technology systems
10. how to maintain and monitor your sales pipeline
11. when to assign the leads to the relevant members of sales team or to yourself
12. your organisation’s processes and procedures for using, storing and maintaining 
information
13. how to use the sales technology system as part of planning and implementation of
sales activities
14. the support, guidance and skills development available to the users of sales tools
and technology system
15. how to seek recommendations for improvements of sales activities using the tools
and technology systems"""

In [14]:
from pydantic import Field
from typing import Dict, List
from helpers._types import (
    Agent,
    StrictTool,
    Result,
)

In [6]:
import os
os.chdir(f"{os.path.dirname(os.path.abspath(os.getcwd()))}/zavmo")

In [1]:
import openai
import json
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
# from IPython.display import Markdown

In [112]:
class BloomTaxonomyLevels(StrictTool):
    level: Literal["Remember", "Understand", "Apply", "Analyze", "Evaluate", "Create"] = Field(description="The level of Bloom's Taxonomy")
    criteria: str = Field(description="Generate specific criteria for assessing the skill based on shared NOS for the level of Bloom's Taxonomy")

    def execute(self, context: Dict):
        return self.model_dump_json()
    
class GetSkillFromNOS(StrictTool):
    skill_area: str = Field(description="Name of the skill area")
    skill_category: str = Field(description="Technical or Soft skill category")
    blooms_taxonomy_criteria: List[BloomTaxonomyLevels] = Field(description="Criteria for Bloom's Taxonomy levels")
    
    def execute(self, context: Dict):
        return self.model_dump_json()

class GetSkillsFromNOS(StrictTool):
    """A tool to extract multiple skills names, categories and assessment criteria from the NOS document."""
    
    nos: List[GetSkillFromNOS] = Field(description="List of skills with categories and criteria on Bloom's Taxonomy levels")
    
    def execute(self, context: Dict):
        return self.model_dump_json()


In [113]:
from helpers.swarm import fetch_agent_response


In [114]:
messages = [{"role": "user", "content": f"Here is the NOS(National Occupational Standards) document: {nos}"}]
agent = Agent(
    name="GetSkillsFromNOS",
    functions=[GetSkillsFromNOS],
    model="gpt-4o-mini",
    system_message=system_message,
)

agent.instructions = sys

response = fetch_agent_response(agent, messages, {})  

2024-12-02 23:46:50,108 - INFO - Logging from fetch_agent response


In [115]:
response_content = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
response_content


{'nos': [{'skill_area': 'Sales Planning and Implementation',
   'skill_category': 'Technical',
   'blooms_taxonomy_criteria': [{'level': 'Remember',
     'criteria': 'Recall the types of sales-related information required for effective planning.'},
    {'level': 'Understand',
     'criteria': 'Explain the importance of using sales-related information in planning and implementing activities.'},
    {'level': 'Apply',
     'criteria': 'Use sales-related information to develop a sales plan.'},
    {'level': 'Analyze',
     'criteria': 'Identify components of an effective sales plan based on sales data.'},
    {'level': 'Evaluate',
     'criteria': 'Assess the effectiveness of a sales plan based on sales outcomes.'},
    {'level': 'Create',
     'criteria': 'Design a comprehensive sales plan that incorporates various sales-related information.'}]},
  {'skill_area': 'Sales Technology and Information Management',
   'skill_category': 'Technical',
   'blooms_taxonomy_criteria': [{'level': 'Re

In [116]:
with open(os.path.join(os.getcwd(), "assets/nos/sales/INSSAL014.json"), 'w') as file:
    json.dump(response_content['nos'], file, indent=2)

In [117]:
def get_nos_skills_with_criteria():
    nos_skills = response_content['nos']
    skill_criteria_str = ""
    for skill in nos_skills:
        skill_name = skill.get("skill_area")
        criteria = skill.get("blooms_taxonomy_criteria")
        criteria_str = "\n"+"\n".join([f"{item['level']}: {item['criteria']}" for item in criteria])
        skill_criteria_str += f"Skill: {skill_name}\nCriteria: {criteria_str}\n\n"
    return skill_criteria_str

In [118]:
print(get_nos_skills_with_criteria())

Skill: Sales Planning and Implementation
Criteria: 
Remember: Recall the types of sales-related information required for effective planning.
Understand: Explain the importance of using sales-related information in planning and implementing activities.
Apply: Use sales-related information to develop a sales plan.
Analyze: Identify components of an effective sales plan based on sales data.
Evaluate: Assess the effectiveness of a sales plan based on sales outcomes.
Create: Design a comprehensive sales plan that incorporates various sales-related information.

Skill: Sales Technology and Information Management
Criteria: 
Remember: Recall the key features of the sales technology systems used.
Understand: Explain how to check the quality, validity, and reliability of stored information.
Apply: Demonstrate how to store information on sales technology systems.
Analyze: Evaluate stored data for its accuracy and relevance.
Evaluate: Make informed decisions based on the analysis of the informatio

In [24]:
url = 'http://127.0.0.1:8000/api/create_org/'


In [25]:
payload = {
    "org_name": "Kenpath"
}

In [26]:
response = requests.post(url, json=payload)

In [27]:
response

<Response [201]>

In [34]:
url = 'http://127.0.0.1:8000/api/create_user/'  
data = {
    'email': 'smrithi.shaji@kenpath.io',
    'org_id': '2'  
}

response = requests.post(url, json=data)
print(response.status_code)
print(response.json())


400
{'org_id': ['This field is required.']}
